In [71]:
# import libraries
import pandas as pd
import numpy as np
import json
from IPython.display import JSON
import nltk
import re
import string
from tqdm import tqdm
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from scipy.sparse import save_npz, load_npz

# import own functions
import preprocessing

### Parse json files containing questions from consumers and experts

In [ ]:
filepath = 'data/EPIC-QA/consumer_questions_primary.json'
consumer_df = preprocessing.parse_questions(filepath)
consumer_df.head()

In [ ]:
# consumer_df.to_csv('data/processed/questions_consumer.csv', sep='\t')

In [ ]:
# parse json files containing questions from consumers and experts
filepath = 'data/EPIC-QA/expert_questions_primary.json'
expert_df = preprocessing.parse_questions(filepath)
expert_df.head()

In [ ]:
# expert_df.to_csv('data/processed/questions_expert.csv', sep='\t')

### Extract titles and abstracts from metadata file

In [2]:
metadata = preprocessing.load_metadata('data/cord-19/metadata.csv')

CSV file loaded successfully
Metadata cleaning complete


In [3]:
# Create table of just cord_uid, title, and abstracts
content = metadata[['cord_uid', 'title', 'abstract']]

In [4]:
df = preprocessing.make_title_abstract_documents(content)

100%|██████████| 536570/536570 [28:47<00:00, 310.62it/s] 
/mnt/c/Users/omlea/Documents/_training/Lighthouse/lh-final-project/covid-lit-search/preprocessing.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title_abstract'] = l


In [5]:
df.head()

,cord_uid,title_abstract
0,ug7v899j,clinical feature culture prove mycoplasma pneu...
1,02tnwd4m,nitric oxide pro inflammatory mediator lung di...
2,ejv2xln0,surfactant protein d pulmonary host defense su...
3,2b73a28n,role endothelin lung disease endothelin et ami...
4,9785vg6d,gene expression epithelial cells response pneu...


In [6]:
# df.to_csv('data/processed/titles_abstracts_merged_cleaned.csv')

### Remove non-english

In [10]:
df = pd.read_csv('data/processed/titles_abstracts_merged_cleaned.csv', index_col=0)

langs = pd.read_csv('data/processed/languages.csv', index_col=0)['0']
df['language'] = langs
df = df[df.language == 'en'].drop(columns='language')

In [11]:
# df.to_csv('data/processed/titles_abstracts_merged_cleaned.csv')

## Vectorization 

In [50]:
# load data from file
df = pd.read_csv('data/processed/titles_abstracts_merged_cleaned.csv', index_col=0)

In [51]:
df.head()

,cord_uid,title_abstract
0,ug7v899j,clinical feature culture prove mycoplasma pneu...
1,02tnwd4m,nitric oxide pro inflammatory mediator lung di...
2,ejv2xln0,surfactant protein d pulmonary host defense su...
3,2b73a28n,role endothelin lung disease endothelin et ami...
4,9785vg6d,gene expression epithelial cells response pneu...


In [52]:
# drop null rows
df.dropna(inplace=True)

In [56]:
def clean_alphanumeric(s):
    """Drops words containing more than 4 digits, or beginning with digits then letters"""
    s = re.sub(r'\d[a-z]*\d[a-z]*\d[a-z]*\d[a-z]*\d[\d\w]*', '', s)
    s = re.sub(r'\d+[a-z]+\W*', '', s)
    s = " ".join(word.strip() for word in s.split())
    return s

df['title_abstract'] = df['title_abstract'].apply(clean_alphanumeric)

In [57]:
tfidf = TfidfVectorizer()
tfidf.fit(df.title_abstract)
X = tfidf.transform(df.title_abstract)

In [68]:
# # save fit vectorizer and term-document matrix
# with open('data/processed/vectorizer_1.pkl', 'wb') as file:
#     pickle.dump(tfidf, file)

In [70]:
# save_npz('data/processed/tdm1.npz', X)